### Documents
LangChain implements a Document abstraction which is intended to represent a unit of text and associated metadata. It has two attributes:

* page_content: a string representing the content
* metadata: a dict containing arbitrary metadata. The metadata attribute can capture information about the 
    source of the document, its relationship to other documents, and other information. 
    Note that an individual Document object often represents a chunk of a larger document.

In [3]:
### Documents
from langchain_core.documents import Document

documents = [
    Document(
      page_content="Dogs are great companions, known for their loyalty and friendliness.",
      metadata={"source": "mammal-pets-doc"},
    ),
    Document(
      page_content="Cats are independent pets that often enjoy their own space.",
      metadata={"source": "mammal-pets-doc"},
    ),
    Document(
      page_content="Goldfish are popular for beginners, requiring relatively simple care.",
      metadata={"source": "fish-pets-doc"},
    ),
    Document(
      page_content="Parrots are intelligent birds capable of mimicking human speech.",
      metadata={"source": "bird-pets-doc"},
    ),
    Document(
      page_content="Rabbits are socian animals that need plenty of space to hop around.",
      metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are socian animals that need plenty of space to hop around.')]

In [5]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGING_FACE"] = os.getenv("HUGGING_FACE")

llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10d20d110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d346b50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/ibrahimkhalilov/miniconda3/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
## VectorStores
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents, embeddings)


In [9]:
vectorstore

In [10]:
vectorstore.similarity_search("cat")

[Document(id='4c123964-c033-481f-8a32-05fc1005ac4f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='bb26463c-91f7-4680-aae8-804824eddb31', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='dcfe324b-da9e-459f-a97a-2f1c660ccc3d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are socian animals that need plenty of space to hop around.'),
 Document(id='521363a7-8416-4323-9bf2-8d56427bf4ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [11]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='4c123964-c033-481f-8a32-05fc1005ac4f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='bb26463c-91f7-4680-aae8-804824eddb31', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='dcfe324b-da9e-459f-a97a-2f1c660ccc3d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are socian animals that need plenty of space to hop around.'),
 Document(id='521363a7-8416-4323-9bf2-8d56427bf4ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [12]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='4c123964-c033-481f-8a32-05fc1005ac4f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058006286621),
 (Document(id='bb26463c-91f7-4680-aae8-804824eddb31', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='dcfe324b-da9e-459f-a97a-2f1c660ccc3d', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are socian animals that need plenty of space to hop around.'),
  1.5879638195037842),
 (Document(id='521363a7-8416-4323-9bf2-8d56427bf4ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

In [13]:
### Retrievers
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])


[[Document(id='4c123964-c033-481f-8a32-05fc1005ac4f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='bb26463c-91f7-4680-aae8-804824eddb31', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

### Better way to do the same:

In [16]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='4c123964-c033-481f-8a32-05fc1005ac4f', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='bb26463c-91f7-4680-aae8-804824eddb31', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")

print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
